In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex
df = pd.read_csv('responses_scrubbed.csv', index_col=None, parse_dates=['date'])

## Number of respondents from each neighborhood
Gives us a sense of who responded.

In [2]:
df['neighborhood'].value_counts()

Harmon                 151
Upper Village          107
Old Post Road North     68
Mount Airy              55
Sunset Park             42
Albany Post Road        29
Half Moon Bay           19
Quaker Ridge             9
Cortlandt                8
Ossining                 1
Name: neighborhood, dtype: int64

## Number of respondents from each demographic
Gives us a sense of who responded.

In [3]:
df['demographic'].value_counts()

Adult with school kid       194
Adult without school kid    165
Senior                      128
College kid                   5
High School kid               3
Middle School kid             1
Name: demographic, dtype: int64

## Percent respondents of each demographic from each neighborhood
Each column adds up to 100%.

In [4]:
df2 = df.groupby(['demographic', 'neighborhood']).size().unstack(fill_value=0).apply(lambda x: 100 * x / x.sum()) #.astype(int)
df2.round(0).astype(int) # round to int

neighborhood,Albany Post Road,Cortlandt,Half Moon Bay,Harmon,Mount Airy,Old Post Road North,Ossining,Quaker Ridge,Sunset Park,Upper Village
demographic,,,,,,,,,,
Adult with school kid,21,25,5,40,47,46,0,44,43,42
Adult without school kid,28,50,47,27,29,36,100,33,40,34
College kid,0,0,0,2,0,1,0,0,2,0
High School kid,0,0,0,1,2,0,0,0,0,0
Middle School kid,0,0,0,1,0,0,0,0,0,0
Senior,52,25,47,29,22,16,0,22,14,25


## Set up some useful functions for talllying responses

In [5]:
# splits the comma-separated values in the column into a pandas Series
def split_by_comma(group, column_title):
    try:
        value_list = group[column_title].str.split(',')
        return pd.Series(value_list.values[0], name=column_title)
    except:
        return []

# breaks up the indicated column representing checkbox type data into multiple columns
def get_answer_options(column_title, df, series_name):
    # group all respondents who gave the same child_drive_reason
    ser = df[pd.notnull(df[column_title])].groupby(level=0).apply(split_by_comma, column_title=column_title)
    try:
        # remove extra whitespace, if any responses exist
        ser = ser.map(lambda x: str(x).strip()) #strip whitespace from answers
    except:
        # there were no results in the series
        pass
    
    dum = pd.get_dummies(ser) # split each respondent's answers into separate columns

    # sum columns
    col_list = list(dum.columns) 
    dum.loc[series_name] = dum[col_list].sum(axis=0)
    
    # sort and return only the top 10
    return dum.loc[series_name].sort_values(ascending=False)[0:10]

# entry point
def get_responses(column_title, df, demographic='Anyone', neighborhood='Anywhere', as_percent=True, series_name='overall'):
    # how many people gave reasons of any kind?
    num_respondents = df[pd.notnull(df[column_title])].shape[0]
    
    # break up the column of interest
    results = get_answer_options(column_title, df, series_name)
    
    # calculate percentages, if requested
    if as_percent:
        results = results.apply(lambda x: int(100 * x / num_respondents))

    return results


# Analyze responses

In [6]:
# a list of multiple-choice columns we'd like to analyze
checkbox_fields = [ 'schools', 'child_bus_freq', 'child_walk_freq', 'child_driven_freq', 'child_drive_freq', 'child_bike_freq', 'child_drive_reason', 'child_no_walk_reason', 'child_no_bike_reason', 'walk_freq', 'bike_freq', 'bikes_on_sidewalk', 'self_jog_frequency', 'commutes', 'child_self_school', 'child_self_bus_freq', 'child_self_bike_freq', 'child_self_driven_freq', 'child_self_drive_freq', 'child_self_walk_freq', 'child_self_commutes', 'child_self_has_children', 'commuter_distance', 'commuter_type', 'commuter_walk_to_station_freq', 'commuter_bike_to_station_freq', 'commuter_drive_to_station_freq', 'commuter_carpool_to_station_freq', 'commuter_driven_to_station_freq', 'commuter_bus_to_station_freq', 'no_walk_reason', 'no_bike_reason', 'drive_reason', 'drivers_are_safe', 'bicyclists_are_safe', 'contact_interest' ]

# builds a set of columns representing each demographic's 
def break_down_by(column_title, breakdown_column='demographic'):
    
    # get the overall reasons people gave for each
    responses = get_responses(column_title, df, series_name='overall')
    # convert to dataframe
    df2 = responses.to_frame()
    
    # add on columns for each demographic to the dataframe
    for breakdown_index, v in df[breakdown_column].value_counts().items():
        df3 = df[df[breakdown_column] == breakdown_index ]
        try:
            # get the reasons this specific demographic gave for this column
            responses = get_responses(column_title, df3, breakdown_index, series_name=breakdown_index)
            df2 = pd.concat([df2, responses.to_frame()], axis=1)
            #print(df2)
        except:
            # there were no responses from this demographic
            continue
            
    return df2

        
# loop through each of the checkbox-type questions
for column_title in checkbox_fields:
    
    # break this column down by demographic
    df2 = break_down_by(column_title, breakdown_column='demographic')
    
    # break this column down by demographic
    df3 = break_down_by(column_title, breakdown_column='neighborhood')
    
    # output top 10 results
    num_respondents = df[pd.notnull(df[column_title])].shape[0] # get the number of people who answered this question
    display(Markdown('## *{}* (as % of {} respondents):'.format(column_title, num_respondents).title()))

    display(Markdown('### Broken down by demographic:'.title()))
    print(df2)
    #display(Markdown('### Broken down by neighborhood:'.title()))
    #print(df3)
    

## *Schools* (As % Of 188 Respondents):

### Broken Down By Demographic:

                                   overall  Adult with school kid
CET                                     45                     45
PVC                                     31                     31
CHHS                                    30                     30
Homeschooled                             2                      2
Circle School                            1                      1
St. Augustine                            1                      1
Happy Hearts                             1                      1
The Harvey School                        0                      0
Currently pre-school but soon CET        0                      0
Happy Hearts Day Care                    0                      0


## *Child_Bus_Freq* (As % Of 193 Respondents):

### Broken Down By Demographic:

...

### Broken Down By Demographic:

    overall  Adult with school kid  Adult without school kid  Senior  \
[]      100                    100                       100     100   

    College kid  High School kid  Middle School kid  
[]          100              100                100  


## *Contact_Interest* (As % Of 457 Respondents):

### Broken Down By Demographic:

       overall  Adult with school kid  Adult without school kid  Senior  \
0_No        54                     52                        53      55   
0_Yes       45                     47                        46      44   

       College kid  High School kid  Middle School kid  
0_No            60            100.0              100.0  
0_Yes           40              NaN                NaN  


## Who rides on the sidewalk?

### Overall

In [7]:
# do people generally bike on the sidewalk?
df['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,49
No - I only bicycle on the road,29
Yes,23


### People who never bike

In [8]:
# do people who never bike claim to ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Rarely or never']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,80
No - I only bicycle on the road,13
Yes,7


### People who bike at least occasionally

In [9]:
# do people who ride a bike occasionally or more ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days', 'Once in a while']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I don't bicycle,1
No - I only bicycle on the road,53
Yes,46


### People who bike a lot

In [10]:
# do people who bike a lot ride on the sidewalk?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day']) ]
df2['bikes_on_sidewalk'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bikes_on_sidewalk
No - I only bicycle on the road,58
Yes,42


## Who thinks drivers are safe?

### Overall

In [11]:
# do people generally think drivers are safe?
df['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()

,drivers_are_safe
1.0,9
2.0,30
3.0,47
4.0,14


### People who walk a lot

In [12]:
# do people who walk a lot think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,11
2.0,32
3.0,46
4.0,10


### People who rarely walk

In [22]:
# do people who rarely walk think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,6
2.0,25
3.0,46
4.0,22


### People who never walk

In [14]:
# do people who never walk think drivers are safe?
df2 = df[ df['walk_freq'].isin(['Rarely or never']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,10
2.0,22
3.0,42
4.0,25


### People who bike frequently

In [23]:
# do people who bike a lot think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()

,drivers_are_safe
1.0,13
2.0,33
3.0,47
4.0,8


## Who thinks bicyclists are safe?

### Overall

In [15]:
# do people generally think bicyclists are safe?
df['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,13
2.0,23
3.0,49
4.0,15


### People who bike a lot

In [16]:
# do people who bike a lot think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Every day, or almost every day', 'Once every few days']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,5
2.0,15
3.0,57
4.0,24


### People who rarely bike

In [17]:
# do people who never bike think bicyclists are safe
df2 = df[ df['bike_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['bicyclists_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,bicyclists_are_safe
1.0,15
2.0,25
3.0,46
4.0,13


In [19]:
# do people who never bike think drivers are safe?
df2 = df[ df['bike_freq'].isin(['Rarely or never', 'Once in a while']) ]
df2['drivers_are_safe'].value_counts().to_frame().apply(lambda x: 100 * x / x.sum()).round(0).astype(int).sort_index()


,drivers_are_safe
1.0,9
2.0,29
3.0,47
4.0,16


In [20]:
df[df['demographic'] == 'College kid']['walk_freq'].value_counts().sort_index()

Once every few days    2
Once in a while        2
Rarely or never        1
Name: walk_freq, dtype: int64